<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex8-Efficient_Frontier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Foundations of Investment Analysis***, Bates, Boyer, and Fletcher  


# Example 8: The Efficient Frontier



### Imports and Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize

### Define Inputs
In the code cell below we define the covariance matrix and the vector of expected returns.  The default is for the case of three assets, but these can be for any number of $N$ assets. Whatever $N$ is, the `covariance_matrix` is $N\times N$ and the array of `expected_returns` is $N \times 1$.   

In [2]:
covariance_matrix = np.array([[0.0400, -0.0200, 0.0250], [-0.0200, 0.0900, 0.0400], [0.0250, 0.0400, 0.2500]])
expected_returns = np.array([[0.12], [0.06], [0.08]])
rf = 0.05
# check dimensions of inputs
print("N=", expected_returns.shape[0])
print("Dimension of Covariance Matrix:",covariance_matrix.shape)
print("Dimension of Expected Returns:", expected_returns.shape)

N= 3
Dimension of Covariance Matrix: (3, 3)
Dimension of Expected Returns: (3, 1)


### Define Functions to Determine Portfolios on EFRS
In the code cell below, we create two functions to use in determining portfolios on the EFRS. These functions could be stored in a `.py` file and loaded in the "Imports and Setup" section to keep the code more organized (like functions in simple_finance.py).  We define them here so you can see what is going on.

---
**Function1**  
`portfolio_volatility`  
*inputs:*  
1. vector of weights  
2. covariance matrix.  
*output:*  
The output of this function is the volatility of the portfolio using Essential Math Fact #5.  
*example usage:*  
`weights = np.ones(3) / 3` # define weights of equally-weighted portfolio  
`vol = portfolio_volatility(weights, covariance_matrix)` # use function  
`print(vol)`  
`0.22852182001336813`
  
---
**Function 2**  
`EFRS_portfolio`  
*inputs:*  
1. a single target portfolio expected return
2. vector of expected returns for individual assets  
3. covariance matrix.  
*output:*
The vector of weights of the portfolio with minimum volatilty that has the given targe expected return.

*further comments:*  
The function `EFRS_portfolio` uses the `minimize` function from the `scipy.optimize` package. It is Python's version of Excel solver. See code cell below for example usage.

---
**Python's Version of Excel Solver**   
`minimize` imported from `scipy.optimize`  
*inputs:*  
1. a function to mimimize; The variables that will change to mimize the function are always defined by the function's *first* input. See comments in code cell for more details.
2. starting values
3. additional arguments needed for the function to execute
4. minimization method
5. constraints
*output:*  
The variables that minimize the function




In [3]:
# Function 1
def portfolio_volatility(weights, covariance_matrix):
    return np.sqrt(weights.T @ covariance_matrix @ weights)  # Essential Math Fact #5


# Function 2
def EFRS_portfolio(target_return, expected_returns, covariance_matrix):

   # Define constraints (tuple with two dictionaries)
    constraints = (
        # Constraint 1: Ensure portfolio return equals the target return
        # "type": "eq" specifies an equality constraint (must be exactly zero)
        # "fun": defines the function
        {"type": "eq", "fun": lambda w: w.T @ expected_returns - target_return},

        # Constraint 2: Ensure portfolio weights sum to 1 (fully invested)
        # "type": "eq" specifies an equality constraint (must be exactly zero)
        # "fun": defines the function
        {"type": "eq", "fun": lambda w: np.sum(w) - 1},
    )

    # starting values
    initial_weights = np.ones(3) / 3

    # Python's version of Excel Solver
    result = minimize(
        fun=portfolio_volatility, # function to minimize
        x0=initial_weights, # starting values
        args=(covariance_matrix), # additional arguments needed for function
        method="SLSQP", # minimization method
        constraints=constraints, # constraints
        )

    return result.x

### Developing the Vector of Target Returns
In the code cell below we develop the vector of target expected returns. The defulat is to create 100 target expected returns, equally spaced, with the minimum being half the half the lowest individual asset expected return, and the maximum being twice the highest individual asset epected return.

In [ ]:
# Target returns
min_target = 0.5 * min(expected_returns)
max_target = 2 * max(expected_returns)
target_returns = np.linspace(min_target, max_target, num=100)

### Create Portfolios on EFRS
In the code cell below we applu the `EFRS_portfolio` function.

In [ ]:
EFRS = [EFRS_portfolio(target, expected_returns, covariance_matrix) for target in target_returns]

In [ ]:
plt.plot(volatilities, returns, label="EFRS", linewidth=2, color="k", zorder=1)
plt.scatter(vol1, Er1, s=30, color="black", zorder=2)
plt.xlim(0)
# plt.axhline(y=0, color="k")
plt.xlabel("Volatility")
plt.ylabel("Expected Return")
# plt.title("Efficient Frontier")
plt.legend()
plt.savefig("plots/CH8_8.2.4_efficient_frontier.png", dpi=300)
plt.show()